# Signature detection

This notebook contains experiments for detecting & capturing signatures in document.

In [1]:
import os
import cv2
import numpy as np
import supervision as sv
from pathlib import Path
import fitz
from datetime import datetime
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import torch

print(f"PyTorch version: {torch.__version__}")

if torch.cuda.is_available():
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}: {torch.cuda.device_count()} GPU(s)")
else:
    print("CUDA is not available. Using CPU.")
    print(f"{torch.cpu.device_count()} CPU core(s) available")

PyTorch version: 2.8.0+cu129
CUDA is available. Using GPU: NVIDIA GeForce RTX 5080 Laptop GPU: 1 GPU(s)


## YOLO v8 Signature detector (Fine-tuned)

Link: https://huggingface.co/tech4humans/yolov8s-signature-detector

In [ ]:
# Download yolov8-signature model from huggingface
from ultralytics import YOLO
from huggingface_hub import hf_hub_download, login

login(os.getenv("HF_TOKEN"))

In [ ]:
yolo_model_path = hf_hub_download(
  repo_id="tech4humans/yolov8s-signature-detector",
  filename="yolov8s.pt"
)
print(yolo_model_path)

NameError: name 'hf_hub_download' is not defined

In [ ]:
yolo_model = YOLO(yolo_model_path)

In [ ]:
# Process PDF as image and save it
def process_pdf(pdf_path):
    """
    Process a PDF file to detect signatures and return cropped images with page numbers.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        list: List of tuples (cropped_image, page_number)
    """
    detected_signatures = []
    doc = fitz.open(pdf_path)

    for page_num in range(doc.page_count):
        page = doc[page_num]
        pix = page.get_pixmap()
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
        results = yolo_model(img)
        detections = sv.Detections.from_ultralytics(results[0])

        # Filter detections with confidence > 0.25
        print(f"Confidence: {detections.confidence}")
        detections = detections[detections.confidence > 0.75]
        if len(detections) > 0:
            # Access the bounding boxes using the xyxy property
            for xyxy in detections.xyxy:
                print(f"Has detected signatures")
                # Convert coordinates to integers
                x1, y1, x2, y2 = map(int, xyxy)

                # Ensure coordinates are within image bounds
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(img.shape[1], x2)
                y2 = min(img.shape[0], y2)

                cropped_image = img[y1:y2, x1:x2]

                # Add tuple of (cropped_image, page_number) to the list
                detected_signatures.append((cropped_image, page_num + 1))

    doc.close()
    return detected_signatures

def save_detected_signatures(pdf_path, base_output_dir):
    """
    Process PDF and save detected signatures with page-based subfolder organization.

    Args:
        pdf_path (str): Path to the PDF file
        base_output_dir (str): Base directory to save the signatures

    Returns:
        str: Path to the created output directory
    """
    # Get PDF filename without extension
    pdf_name = Path(pdf_path).stem

    # Create timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Create main subfolder with PDF name and timestamp
    pdf_dir = os.path.join(
        base_output_dir,
        f"{pdf_name}_{timestamp}"
    )

    # Process the PDF
    signatures = process_pdf(pdf_path)

    # Group signatures by page number
    signatures_by_page = {}
    for signature, page_num in signatures:
        if page_num not in signatures_by_page:
            signatures_by_page[page_num] = []
        signatures_by_page[page_num].append(signature)

    # Save signatures organized by page
    for page_num, page_signatures in signatures_by_page.items():
        # Create page subfolder
        page_dir = os.path.join(pdf_dir, f"page_{page_num:03d}")
        os.makedirs(page_dir, exist_ok=True)

        # Save each signature in the page
        for idx, signature in enumerate(page_signatures):
            output_path = os.path.join(
                page_dir,
                f'signature_{idx:03d}.png'
            )
            cv2.imwrite(output_path, signature)
            print(f"Saved signature {idx} from page {page_num} to {output_path}")

    return pdf_dir

In [ ]:
# Example usage:
# Process a PDF and save all detected signatures
pdf_path = "../../data/contracts/Contract291936Van_Gobbel.pdf"
base_output_dir = "./outputs/02-signature-detection/detected_signatures/yolo"

output_folder = save_detected_signatures(pdf_path, base_output_dir)

print(f"All signatures saved in: {output_folder}")


0: 640x480 (no detections), 95.5ms
Speed: 42.7ms preprocess, 95.5ms inference, 37.9ms postprocess per image at shape (1, 3, 640, 480)
Confidence: []

0: 640x480 (no detections), 16.7ms
Speed: 3.6ms preprocess, 16.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
Confidence: []

0: 640x480 1 signature, 15.0ms
Speed: 3.1ms preprocess, 15.0ms inference, 157.2ms postprocess per image at shape (1, 3, 640, 480)
Confidence: [    0.78662]
Has detected signatures
Saved signature 0 from page 3 to ./outputs/02-signature-detection/detected_signatures\Contract291936Van_Gobbel_20250921_221605\page_003\signature_000.png
All signatures saved in: ./outputs/02-signature-detection/detected_signatures\Contract291936Van_Gobbel_20250921_221605


## Conditional-DETR 50 Model Signature (Fine tuned)

Link: https://huggingface.co/tech4humans/conditional-detr-50-signature-detector

In [ ]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection

processor = AutoImageProcessor.from_pretrained("tech4humans/conditional-detr-50-signature-detector", use_fast=True)
detr_model = AutoModelForObjectDetection.from_pretrained("tech4humans/conditional-detr-50-signature-detector")

In [33]:
# Process PDF as image and save it
def process_pdf(pdf_path):
    """
    Process a PDF file to detect signatures and return cropped images with page numbers.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        list: List of tuples (cropped_image, page_number)
    """
    detected_signatures = []
    doc = fitz.open(pdf_path)

    for page_num in range(doc.page_count):
        page = doc[page_num]
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # Increase resolution
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
        results = processor(images=img, return_tensors="pt")

        with torch.no_grad():
            outputs = detr_model(**results)

        target_sizes = torch.tensor([img.shape[:2]])
        detections = processor.post_process_object_detection(
            outputs, target_sizes=target_sizes, threshold=0.75
        )[0]

        detections_zipped = zip(detections["scores"], detections["labels"], detections["boxes"])

        filtered_detections = [det for det in detections_zipped if det[0].item() > 0.75]

        # After filtering detections
        if len(filtered_detections) > 0:
            for score, label, box in filtered_detections:
                # box is the bounding box in xyxy format (x1, y1, x2, y2)
                x1, y1, x2, y2 = map(int, box.tolist())

                # Ensure coordinates are within image bounds
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(img.shape[1], x2)
                y2 = min(img.shape[0], y2)

                cropped_image = img[y1:y2, x1:x2]

                # Add tuple of (cropped_image, page_number) to the list
                detected_signatures.append((cropped_image, page_num + 1))

        for score, label, box in zip(detections["scores"], detections["labels"], detections["boxes"]):
            print(f"Detected signature (label: {label}) at {box.tolist()}, confidence: {score.item()}")

    doc.close()
    return detected_signatures

def save_detected_signatures(pdf_path, base_output_dir):
    """
    Process PDF and save detected signatures with page-based subfolder organization.

    Args:
        pdf_path (str): Path to the PDF file
        base_output_dir (str): Base directory to save the signatures

    Returns:
        str: Path to the created output directory
    """
    # Get PDF filename without extension
    pdf_name = Path(pdf_path).stem

    # Create timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Create main subfolder with PDF name and timestamp
    pdf_dir = os.path.join(
        base_output_dir,
        f"{pdf_name}_{timestamp}"
    )

    # Process the PDF
    signatures = process_pdf(pdf_path)

    # Group signatures by page number
    signatures_by_page = {}
    for signature, page_num in signatures:
        if page_num not in signatures_by_page:
            signatures_by_page[page_num] = []
        signatures_by_page[page_num].append(signature)

    # Save signatures organized by page
    for page_num, page_signatures in signatures_by_page.items():
        # Create page subfolder
        page_dir = os.path.join(pdf_dir, f"page_{page_num:03d}")
        os.makedirs(page_dir, exist_ok=True)

        # Save each signature in the page
        for idx, signature in enumerate(page_signatures):
            output_path = os.path.join(
                page_dir,
                f'signature_{idx:03d}.png'
            )
            cv2.imwrite(output_path, signature)
            print(f"Saved signature {idx} from page {page_num} to {output_path}")

    return pdf_dir

In [34]:
# Example usage:
# Process a PDF and save all detected signatures
pdf_path = "../../data/contracts/Contract291936Van_Gobbel.pdf"
base_output_dir = "./outputs/02-signature-detection/detected_signatures/detr"

# process_pdf(pdf_path)

output_folder = save_detected_signatures(pdf_path, base_output_dir)

# print(f"All signatures saved in: {output_folder}")

Detected signature (label: 0) at [484.26519775390625, 500.1751708984375, 650.2720336914062, 620.5584106445312], confidence: 0.8722892999649048
Saved signature 0 from page 3 to ./outputs/02-signature-detection/detected_signatures/detr\Contract291936Van_Gobbel_20250921_234057\page_003\signature_000.png
